In [109]:
from collections.abc import Iterable

import pymongo
from collections import Counter

In [110]:
c = pymongo.MongoClient()

In [111]:
db0 = c.neuro1337
db1 = c.neuro1338
db2 = c.neuro1339

In [112]:
class CoolDict(dict):
    def __getattr__(self, name):
        attr = self[name]
        if type(attr) == dict:
            return CoolDict(attr)
        return attr


class Block(CoolDict):
    def __init__(self, arg, db=db0):
        self.db = db
        if type(arg) == dict:
            super().__init__(arg)
        elif type(arg) == str:
            super().__init__(db.blocks.find_one({"block.header.id.data": arg}))
        elif type(arg) == int:
            super().__init__(db.blocks.find_one({"block.header.height": arg}))

    @property
    def previousAssembly(self):
        return Assembly(self.previousAssemblyId.data, db=self.db)
    
    @property
    def previousBlock(self):
        return Block(self.block.header.previousBlockHash.data, db=self.db)

    @property
    def transactions(self):
        result = []
        for t in self.db.transactions.find(
            {"blockId.data": self.block.header.id.data, "isCoinbase": False}
        ):
            result.append(Transaction(t))
        return result

    @property
    def coinbase(self):
        return Transaction(
            self.db.transactions.find_one(
                {"blockId.data": self.block.header.id.data, "isCoinbase": True}
            )
        )


class Assembly(CoolDict):
    def __init__(self, arg, db=db0):
        self.db = db
        if type(arg) == str:
            super().__init__(db.assemblies.find_one({"id.data": arg}))
        elif type(arg) == int:
            super().__init__(db.assemblies.find_one({"height": arg}))

    @property
    def previousAssembly(self):
        return Assembly(self.previousAssemblyId.data, db=self.db)


class Transaction(CoolDict):
    def __init__(self, arg, db=db0, pool=True):
        self.db = db
        if type(arg) == dict:
            super().__init__(arg)
        else:
            query = {"transaction.id.data": arg}
            if not pool:
                query["blockId"] = {"$exists": True}
            super().__init__(db.transactions.find_one(query))

    @property
    def block(self):
        return Block(self.blockId.data, db=self.db)


class CoolList(list):
    def __getattr__(self, name):
        return CoolList(getattr(x, name) for x in self)


class Transactions(CoolList):
    def __init__(self, arg=None, db=db0, pool=True):
        if type(arg) is str:
            query = {"transaction.id.data": arg}
            if not pool:
                query["blockId"] = {"$exists": True}
        elif type(arg) is dict:
            query = arg
        else:
            query = {}
        transactions = (Transaction(t) for t in db.transactions.find(query))
        super().__init__(transactions)

    @classmethod
    def by_input(cls, transaction_id, output_id, pool=True):
        query = {
            "transaction.inputs": {
                "$elemMatch": {
                    "id.data": transaction_id,
                    "outputId": output_id,
                }
            }
        }
        if not pool:
            query["blockId"] = {"$exists": True}
        return cls(query)


class Blocks(CoolList):
    def __init__(self, arg=None, db=db0):
        self.db = db0
        if isinstance(arg, Iterable):
            super().__init__(arg)
        else:
            if type(arg) == str:
                query = {"block.header.id.data": arg}
            elif type(arg) == int:
                query = {"block.header.height": arg}
            elif type(arg) == dict:
                query = arg
            else:
                query = {}
            super().__init__([Block(b) for b in db.blocks.find(query)])
    
    @classmethod
    def get_tip(cls, db=db0):
        return Block(db.blocks.find().sort("score", pymongo.DESCENDING).next())

In [49]:
Block(0, db=db1).previousAssembly.previousAssembly

{'_id': ObjectId('5cdad1df85699a66c76e77b7'),
 'id': {'type': 'SHA256',
  'data': 'M24Toupl2M0NUoRo2MZEY9KFvpcRq0InXrZIS9bK704='},
 'finishedComputation': True,
 'height': -2,
 'seed': 0,
 'nbAddresses': 2}

In [50]:
[
    (ob.get("branchPath"), ob["block"]["header"]["height"])
    for ob in db2.blocks.find({"branchPath.branchIds": {"$ne": [0]}})
]

[]

In [51]:
[
    (ob.get("branchPath"), ob["block"]["header"]["height"])
    for ob in db0.blocks.find({"branchPath.branchIds": {"$ne": [0]}})
]

[]

In [52]:
heights = [
    [ob["block"]["header"]["height"] for ob in db.blocks.find()]
    for db in (db0, db1, db2)
]

In [53]:
h0 = set(heights[0])
h1 = set(heights[1])
h2 = set(heights[2])

In [56]:
print(h0 - h1, h1 - h0)
print(h0 - h2, h2 - h0)
print(max(h0), len(h0))

set() set()
set() set()
608721 290


In [55]:
Transactions("1UM57qEdvRZ3V1TmMCjOetoH2URLglDPLSnAlEI6dxY=", pool=False).block.branchPath

[]

In [121]:
Counter(Blocks().block.header.author.keyPub.rawData)

Counter({'Awqh/GaK+zXAydO8h4sDf4apFNmW8SFaIqRqXioYDXy8': 101,
         'A+BPdJmNMRu7dZ0O4+b/jG5CyuLeI870VKYu0DrtJ8I8': 93,
         'AyMKGnZhazXAoXb0SJIqi4Ho+lk6bw0UttE34PDtDbya': 74})

In [38]:
Transactions("0eLNKWSfT51YkpVC6CZRWY9BOX2AUZIZseQxvFssCWs=", db=db1, pool=False).block.branchPath

[{'branchIds': [0], 'blockNumbers': [5]}]

In [32]:
Transactions("7002Vjd2bbrN/nA3B1wvjN1vi/ja9lEiSanGs9WEGpU=").block.branchPath

[{'branchIds': [1, 0], 'blockNumbers': [1, 0]}]

In [28]:
Transactions("7002Vjd2bbrN/nA3B1wvjN1vi/ja9lEiSanGs9WEGpU=")

[{'_id': ObjectId('5cda75ef85699a362a5d70c5'),
  'transaction': {'id': {'type': 'SHA256',
    'data': '7002Vjd2bbrN/nA3B1wvjN1vi/ja9lEiSanGs9WEGpU='},
   'inputs': [{'id': {'type': 'SHA256',
      'data': 'BnkJitgML5phxXJQabhzzeBN1PN5bFVEALjwYgnK8cs='},
     'outputId': 1,
     'signatureId': 0},
    {'id': {'type': 'SHA256',
      'data': 'JtHQcoxC2xsVbwyGBfg5tmvJn57QWXJpJRf44KxY4VE='},
     'outputId': 0,
     'signatureId': 0}],
   'outputs': [{'address': {'data': 'N2DJgbU3v8tbbzaDdBEyn5fPnLZVzAFC6E'},
     'value': {'value': '500050'}},
    {'address': {'data': 'NHW276q4ZQvaDTDGWLvnrfrnVsUY18Esjm'},
     'value': {'value': '500050'}}],
   'signatures': [{'signature': {'type': 'SHA256',
      'data': 'kLZhZSIOhAioxJlJea3o0SOgoXZqHEUGf5YnWETpEEL9MTUVfWhJKumBuZ6Nxqa04fPN/kluYnXxtG8X/Y0G0Q=='},
     'keyPub': {'type': 'ECP256K1',
      'rawData': 'A3VGExsTvRZm+9FuzCT9u2aef+L5Fnz8HkH8FdGSlfpq'}},
    {'signature': {'type': 'SHA256',
      'data': 'fHO4Ng9wq2P8QfQvsgPA0snk/Iev5yr+XAl2MEC

In [41]:
Transactions.by_input(transaction_id="pEMC0wWSrVr1boX2lxMte8S/p7LCD/V+0HgbcJ1q7lY=", output_id=1, pool=False)

[{'_id': ObjectId('5cdac1e985699a60a77f7a9e'),
  'transaction': {'id': {'type': 'SHA256',
    'data': '5lpMPp57uSDegM6ig1P3u54xLtGILv3tpF+Nyw4iTwY='},
   'inputs': [{'id': {'type': 'SHA256',
      'data': 'pEMC0wWSrVr1boX2lxMte8S/p7LCD/V+0HgbcJ1q7lY='},
     'outputId': 1,
     'signatureId': 0},
    {'id': {'type': 'SHA256',
      'data': 'auWyK0SH4vHEruX6ANZ4LBITvCTgvP7oz7HFkCrUdes='},
     'outputId': 0,
     'signatureId': 0},
    {'id': {'type': 'SHA256',
      'data': 'Yb0yya4XdhKZpZfW6HbHgU9UHJ4JskL2KXqumsOeRGo='},
     'outputId': 0,
     'signatureId': 0}],
   'outputs': [{'address': {'data': 'N2DJgbU3v8tbbzaDdBEyn5fPnLZVzAFC6E'},
     'value': {'value': '584180'}},
    {'address': {'data': 'NHW276q4ZQvaDTDGWLvnrfrnVsUY18Esjm'},
     'value': {'value': '584181'}}],
   'signatures': [{'signature': {'type': 'SHA256',
      'data': 'BltYjDlefKyGueBzxV8q6mYG+EHZ5ixnIws27/ZmdTZwmy9T+DwUC0bD8vHxESLc1I0bkdow/8zrqtxzevxIHA=='},
     'keyPub': {'type': 'ECP256K1',
      'rawData': 'Ant

In [44]:
[b for b in db0.blocks.find({"block.header.id.data": "f4h1GPiyhebWqZM1IUIsoNfG2hmxOgKYt3OQmJ8QmZc="})]

[{'_id': ObjectId('5cdac1e985699a60a77f7a9c'),
  'block': {'header': {'id': {'type': 'SHA256',
     'data': 'f4h1GPiyhebWqZM1IUIsoNfG2hmxOgKYt3OQmJ8QmZc='},
    'timestamp': {'data': 1557840361},
    'previousBlockHash': {'type': 'SHA256',
     'data': 'gOZDavHsQt/CQx/QkmbC+lnbvUWFUAjzzU3ubwjTXMc='},
    'author': {'signature': {'type': 'SHA256',
      'data': '1tkTS1BtoAkNFAO6NkFtIAbTkaQkElFMRjv+XyrJEAFg7lP34Z0/aqXnsqCizCitxdNUtYOkG8UuKOkEhFUwUw=='},
     'keyPub': {'type': 'ECP256K1',
      'rawData': 'Awqh/GaK+zXAydO8h4sDf4apFNmW8SFaIqRqXioYDXy8'}},
    'height': 607617}},
  'branch': 'UNVERIFIED',
  'branchPath': {'branchIds': [0], 'blockNumbers': [125]}}]

In [22]:
Block("KhlcPujzAs58+e9LT/2XeNueJEbZEvT4efAAtnLD6Uk=")

{'_id': ObjectId('5cc9969e85699a12a228294d'),
 'block': {'header': {'id': {'type': 'SHA256',
    'data': 'KhlcPujzAs58+e9LT/2XeNueJEbZEvT4efAAtnLD6Uk='},
   'timestamp': {'data': 1556715166},
   'previousBlockHash': {'type': 'SHA256',
    'data': 'SWAo24vprvsuLKwkpuKHIlyfxB36QEPh0eUV1CnsGpA='},
   'author': {'signature': {'type': 'SHA256',
     'data': 'MQPzbioZt9p9ie0p43cq3/qMiVGaN54coHIPjqQnZ2nQHF/D5xlpQqn+B1F1cjhW6c1WYleHDDPQ7ChgsPB/9g=='},
    'keyPub': {'type': 'ECP256K1',
     'rawData': 'Awqh/GaK+zXAydO8h4sDf4apFNmW8SFaIqRqXioYDXy8'}},
   'height': 382578}},
 'branch': 'UNVERIFIED',
 'branchPath': {'branchIds': [0], 'blockNumbers': [31]}}